In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go

# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz
# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process

/opt/homebrew/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df_101 = pd.read_csv('/Users/ved/Desktop/Capstone/Dataset/indian_food.csv')
df_101.head()

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East


In [3]:
df = pd.read_csv('/Users/ved/Desktop/Capstone/Dataset/Cleaned_Indian_Food_Dataset.csv')
df.head()

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url,Ingredient-count
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...,10
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"tomato,salt,chickpea lentils,green chilli,rice...",https://www.archanaskitchen.com/images/archana...,12
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"salt,rice vermicelli noodles (thin),asafoetida...",https://www.archanaskitchen.com/images/archana...,12
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",https://www.archanaskitchen.com/images/archana...,15
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seed...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"tomato,salt,ginger,red chillies,curry,asafoeti...",https://www.archanaskitchen.com/images/archana...,12


In [4]:
# Total number of recipes by state from Indian Food 101 dataset
df_101.state.value_counts()

Gujarat            35
Punjab             32
Maharashtra        30
West Bengal        24
-1                 24
Assam              21
Tamil Nadu         20
Andhra Pradesh     10
Uttar Pradesh       9
Kerala              8
Odisha              7
Karnataka           6
Rajasthan           6
Telangana           5
Bihar               3
Goa                 3
Manipur             2
Jammu & Kashmir     2
Madhya Pradesh      2
Uttarakhand         1
Tripura             1
Nagaland            1
NCT of Delhi        1
Chhattisgarh        1
Haryana             1
Name: state, dtype: int64

In [5]:
# Extract only recipes belong to Tamil Nadu
df_guj = df_101[df_101['state'] == 'Gujarat']
df_guj

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
56,Basundi,"Sugar, milk, nuts",vegetarian,10,35,sweet,dessert,Gujarat,West
58,Doodhpak,"Milk, rice, sugar, dry fruits",vegetarian,10,60,sweet,dessert,Gujarat,West
168,Saath,"Urad dal, potatoes, wheat flour, sooji",vegetarian,20,40,spicy,main course,Gujarat,West
169,Bajri no rotlo,"Wheat flour, pearl millet flour, hot water",vegetarian,10,10,spicy,main course,Gujarat,West
174,Chevdo,"Citric acid, fry, raisins, sugar, chana daal",vegetarian,20,30,spicy,snack,Gujarat,West
175,Chorafali,"Urad dal, bengal gram flour, dried mango, baki...",vegetarian,5,15,spicy,snack,Gujarat,West
176,Copra paak,"Condensed milk, nestle cream, coconut ice, red...",vegetarian,20,30,-1,main course,Gujarat,West
177,Daal Dhokli,"Whole wheat flour, dal, kokum, gur, bengal gra...",vegetarian,20,30,spicy,main course,Gujarat,West
178,Kutchi dabeli,"Pav, aloo, peanut, pomegranate, star anise",vegetarian,30,10,spicy,snack,Gujarat,West
181,Dhokla,"Rava, coconut, gram flour, mustard, sesame",vegetarian,10,20,spicy,snack,Gujarat,West


User will have a choice where user will select state and there will be the list of state recipies and user can click on  anyone of it and the reult from 101 will be displayed and to get simlar items from 6000 dataset, click on similar items 

In [6]:
# also try below code for ingridients matching

![Alt text](image.png)

In [13]:
# Choosing a dish from 101 dataset and extract different varieties of the same from 6000 dataset
# Use process to extract top 20 matches for the recipe Basundi
possibilities = process.extract("Pumpkin", df['Cleaned-Ingredients'], limit=20, scorer=fuzz.partial_token_sort_ratio)

The process.extract function is used to perform fuzzy string matching and returns a list of possible matches based on the partial ratio similarity score.

It returns list of tuple consisting of three values i.e. recipie name, partial match score and the index value in dataset

In [14]:
possibilities

[('salt,wheat flour,spinach,red chilli powder,sunflower oil rotis,cumin seeds (jeera),parangikai pumpkin (red preferably),coriander powder,water dough,turmeric powder',
  100,
  60),
 ('salt,coconut,mustard seeds,vellai poosanikai (ash gourdwhite pumpkin),water,dry red chilli,curd,curry leaves,coconut oil,methi seeds (fenugreek seeds),black pepper powder,turmeric powder,green chillies',
  100,
  80),
 ('dates (dried s),pine nuts,sunflower oil,nutmeg powder,dried raspberries,wheat flakes (cereal),pumpkin seeds,badam (almond),raisins (sultana),vanilla,walnuts,instant oats (oatmeal),cinnamon powder (dalchini)',
  100,
  172),
 ('tomato,salt,parangikai pumpkin,spaghetti pasta,basil leaves,cloves garlic,virgin olive oil,red chilli flakes,italian,onion,black pepper powder,parmesan cheese',
  100,
  312),
 ('salt,parangikai pumpkin,amchur (dry mango powder),coriander (dhania) leaves,fennel seeds (saunf),black peppercorns,sugar,green chillies,sunflower oil',
  100,
  348),
 ('salt,parangikai p

In [9]:
# Get matches with partial ratio as 100 it does it by comparing all the tuples 2nd element i.e. partial score
# Partial rtaio compare subsections of the string
extracted_matches = [possible for possible in possibilities if possible[1] == 100]
extracted_matches

[]

In [10]:
# Extract only the recipe name from the result
get_recipe_name = [extracted_matches[i][0] for i in range(len(extracted_matches))]
# Get recipes of the extracted matches
df_updated = df[df['TranslatedRecipeName'].isin(get_recipe_name)]

In [11]:
df_updated

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url,Ingredient-count


Select meal based on prep time

In [12]:
a=sns.barplot(y='TranslatedRecipeName',x='TotalTimeInMins',data=df_updated)

ValueError: min() arg is an empty sequence

In [ ]:
print("What's the time you can wait before eating?")
print("1.Instant\n\n2.Can Wait\n\nEnter the number")
x=int(input())
if x==1:
    print(df_updated['TranslatedRecipeName'][df_updated[df_updated['TotalTimeInMins'] <= 40].index.tolist()].tolist())
else:
    print(df_updated['TranslatedRecipeName'][df_updated[df_updated['TotalTimeInMins'] > 40].index.tolist()].tolist())

What's the time you can wait before eating?
1.Instant

2.Can Wait

Enter the number
['Basundi / Rabri Recipe']


In [ ]:
df_guj['ingredients'] = df_guj['ingredients'].to_list()

/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_25011/1323884235.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_guj['ingredients'] = df_guj['ingredients'].to_list()


In [ ]:
print(type(df_guj['ingredients']))

<class 'pandas.core.series.Series'>


In [ ]:
# Explore commonly used ingredient in Gujarati food
df_guj['ingredients'] = df_guj['ingredients'].str.split(',')
df_guj

/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_25011/505738311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_guj['ingredients'] = df_guj['ingredients'].str.split(',')


,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
56,Basundi,"[Sugar, milk, nuts]",vegetarian,10,35,sweet,dessert,Gujarat,West
58,Doodhpak,"[Milk, rice, sugar, dry fruits]",vegetarian,10,60,sweet,dessert,Gujarat,West
168,Saath,"[Urad dal, potatoes, wheat flour, sooji]",vegetarian,20,40,spicy,main course,Gujarat,West
169,Bajri no rotlo,"[Wheat flour, pearl millet flour, hot water]",vegetarian,10,10,spicy,main course,Gujarat,West
174,Chevdo,"[Citric acid, fry, raisins, sugar, chana d...",vegetarian,20,30,spicy,snack,Gujarat,West
175,Chorafali,"[Urad dal, bengal gram flour, dried mango, ...",vegetarian,5,15,spicy,snack,Gujarat,West
176,Copra paak,"[Condensed milk, nestle cream, coconut ice, ...",vegetarian,20,30,-1,main course,Gujarat,West
177,Daal Dhokli,"[Whole wheat flour, dal, kokum, gur, benga...",vegetarian,20,30,spicy,main course,Gujarat,West
178,Kutchi dabeli,"[Pav, aloo, peanut, pomegranate, star anise]",vegetarian,30,10,spicy,snack,Gujarat,West
181,Dhokla,"[Rava, coconut, gram flour, mustard, sesame]",vegetarian,10,20,spicy,snack,Gujarat,West


In [ ]:
df_101['state'].unique()

array(['West Bengal', 'Rajasthan', 'Punjab', 'Uttar Pradesh', '-1',
       'Odisha', 'Maharashtra', 'Uttarakhand', 'Assam', 'Bihar',
       'Andhra Pradesh', 'Karnataka', 'Telangana', 'Kerala', 'Tamil Nadu',
       'Gujarat', 'Tripura', 'Manipur', 'Nagaland', 'NCT of Delhi',
       'Jammu & Kashmir', 'Chhattisgarh', 'Haryana', 'Madhya Pradesh',
       'Goa'], dtype=object)

In [ ]:
# Get unique ingredients with count
plt.figure(figsize=(15,16))
def get_unique_ingredients(state):
    df_unique = df_101[df_101['state'] == state]
    df_unique['ingredients'] = df_unique['ingredients'].str.split(',')
    print(df_unique['ingredients'].explode().unique())
    fig = go.Figure(data=[go.Pie(labels=df_unique['ingredients'].explode().unique(), values=df_unique['ingredients'].explode().value_counts())])
    fig.update_layout(margin=dict(t=0, b=0, l=0, r=0))
    fig.show()
    
get_unique_ingredients('Goa')

['Chicken' ' coconut oil' ' wine vinegar' ' ginger' ' green' ' cinnamon'
 'Coconut milk' ' egg yolks' ' clarified butter' ' all purpose flour'
 'Brown rice' ' fennel seeds' ' grated coconut' ' black pepper'
 ' ginger powder']


/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_25011/713619126.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['ingredients'] = df_unique['ingredients'].str.split(',')


<Figure size 1500x1600 with 0 Axes>

In [ ]:
df['Cuisine'].value_counts()

Continental             952
Indian                  927
North Indian Recipes    763
South Indian Recipes    558
Italian Recipes         231
                       ... 
Burmese                   1
Shandong                  1
Brunch                    1
Lunch                     1
Dessert                   1
Name: Cuisine, Length: 82, dtype: int64